In [1]:
%matplotlib inline

**Read Later:**

Document about ``autograd.Function`` is at
https://pytorch.org/docs/stable/autograd.html#function

**Notes:**

needs to set requires_grad == True when define the tensor if you wanna autograd.

assume z = f(x,y)

z.backward() initiates the backward pass and computed all the gradient automatically.

the gradient will be accumulated into .grad attribute, that means x.grad will have dz/dx and y.grad will have dz/dy.

if z is a scalar, then can use z.backward() and no argument needs to be passed.

if z is a vector, then we have to pass a gradient argument into z.backward(), so if we wanna dz/dx and dz/dy
it should be: z.backward(1). the argument can be any number and should have the same size as z





Autograd: Automatic Differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with
``requires_grad=True``, but for which we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



In [2]:
import torch

Create a tensor and set ``requires_grad=True`` to track computation with it



In [3]:
x = torch.tensor([[1,3],[2,2]], requires_grad=True, dtype = torch.float32)
print(x)
print(x.size())

tensor([[1., 3.],
        [2., 2.]], requires_grad=True)
torch.Size([2, 2])


Do a tensor operation:



In [10]:
y = ( x + 2) 
z = 3 * (y ** 2)
print(y,z)

tensor([[3., 5.],
        [4., 4.]], grad_fn=<AddBackward0>) tensor([[27., 75.],
        [48., 48.]], grad_fn=<MulBackward0>)


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [11]:
print(y.grad_fn)
print(zz.grad_fn)

Do more operations on ``y``



In [12]:
z = y * y # dot product
out = z.mean()
print(z, out)

tensor([[ 9., 25.],
        [16., 16.]], grad_fn=<MulBackward0>) tensor(16.5000, grad_fn=<MeanBackward0>)


``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
flag in-place. The input flag defaults to ``False`` if not given.



In [13]:
a = torch.tensor([2, 2],dtype = torch.float32)
a = ((a * 3))
print(a)
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b)
print(b.grad_fn)

tensor([6., 6.])
False
True
tensor(72., grad_fn=<SumBackward0>)


 Get Gradients
---------
Let's backprop now.
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(1.))``.

Note:
out.backward() only works for scale

out.backward(v) where v is a vector works for non-scale input, it gives Jacobian-vector product. here v is the gradient we have to put into the backwass pass.

In [30]:
# example: y = 2 * x + 3 * (x^2)
x = torch.tensor([3],dtype = torch.float32,requires_grad = False)
x.requires_grad_(True)
y = 2 * x + x * x  * 3
print(x,y)

tensor([3.], requires_grad=True) tensor([33.], grad_fn=<AddBackward0>)


Print gradients d(y)/dx, which should be y =  2 + 6 * x




In [31]:
y.backward() # first run y.backward(), then x.grad has the value
print(x.grad) # only works when x is scalar

tensor([20.])


Mathematically, if you have a vector valued function $\vec{y}=f(\vec{x})$,
then the gradient of $\vec{y}$ with respect to $\vec{x}$
is a Jacobian matrix:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
compute the product $v^{T}\cdot J$. If $v$ happens to be
the gradient of a scalar function $l=g\left(\vec{y}\right)$,
that is,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(Note that $v^{T}\cdot J$ gives a row vector which can be
treated as a column vector by taking $J^{T}\cdot v$.)

This characteristic of vector-Jacobian product makes it very
convenient to feed external gradients into a model that has
non-scalar output.

Note: 

# vector-Jacobian product is matrix dot product!!!

Now let's take a look at an example of vector-Jacobian product:



In [34]:
# example 1:
x = torch.tensor([3,4,6], dtype = torch.float32,requires_grad=True)
y = x * x * 3
v = torch.tensor([1,1,2], dtype=torch.float)
y.backward(v)                                    # here it does the derivative calulation dyi/dxi = 6 * xi * v
print(x.grad)

tensor([18., 24., 72.])


In [41]:
# example 2:
g = torch.tensor([[1,1,1],[2,3,4]],dtype = torch.float32)
g.add_(torch.ones_like(g))
g.requires_grad_(True)         # _ here means replace the variable with new value
print(g, g.requires_grad)
gg = g*g*g
ggg = gg +3*g
v = torch.tensor([[1,1,2],[2,2,2]], dtype=torch.float)
ggg.backward(v)               # here we do the derivative calculation dggg/dg = 3g^2 + 3
print(g.grad)

tensor([[2., 2., 2.],
        [3., 4., 5.]], requires_grad=True) True
tensor([[ 15.,  15.,  30.],
        [ 60., 102., 156.]])


in the above case if we do gg.backward(v), then the derivative is 3g^2 only

In [49]:
x = torch.randn(3, requires_grad=True)
print(x)
y = x * 2
count = 0
while y.data.norm() < 100:    # L2 norm / Euclidean norm
    y = y * 2
    count += 1

print(y,count)

# Now in this case ``y`` is no longer a scalar. ``torch.autograd``
# could not compute the full Jacobian directly, but if we just
# want the vector-Jacobian product, simply pass the vector to
# ``backward`` as argument:
    
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([ 0.7491,  1.0516, -0.0123], requires_grad=True)
tensor([ 95.8882, 134.6020,  -1.5742], grad_fn=<MulBackward0>) 6
tensor([1.2800e+01, 1.2800e+02, 1.2800e-02])


# Stop autograd
You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` either by wrapping the code block in
``with torch.no_grad():``



In [53]:
x = torch.randn(3, requires_grad=True)
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)
    
x = torch.randn(3, requires_grad=True)
x.requires_grad_(False)
print((x**2).requires_grad)

True
True
False
False


Or by using ``.detach()`` to get a new Tensor with the same
content but that does not require gradients:



In [55]:
x.requires_grad_(True)
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y))

True
False
tensor([True, True, True])
